In [1]:
import torch, torchvision
import wandb
import datasets

from utils.model_utils import load_model


c:\Users\Nech\anaconda3\envs\mineralnet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = datasets.load_dataset('Nech-C/mineralimage5K-98')

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'name', 'description', 'mineral_boxes'],
        num_rows: 12828
    })
    validation: Dataset({
        features: ['image', 'name', 'description', 'mineral_boxes'],
        num_rows: 2749
    })
    test: Dataset({
        features: ['image', 'name', 'description', 'mineral_boxes'],
        num_rows: 2749
    })
})

In [3]:
train_ds = ds['train']
test_ds = ds['test']
val_ds = ds['validation']

In [5]:
from utils.image_preprocess import train_preprocess, preprocess

In [6]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x831>,
 'name': 9,
 'description': 'graphic pegmatite.\n\n12x8.5 cm.',
 'mineral_boxes': [{'box': [0.15918, 0.06859, 0.89551, 0.90734],
   'confidence': 0.234,
   'label': 'a stone'}]}

In [7]:
train_ds.set_transform(train_preprocess)
test_ds.set_transform(preprocess)
val_ds.set_transform(preprocess)

In [8]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x831>,
 'name': 9,
 'description': 'graphic pegmatite.\n\n12x8.5 cm.',
 'mineral_boxes': [{'box': [0.15918, 0.06859, 0.89551, 0.90734],
   'confidence': 0.234,
   'label': 'a stone'}],
 'pixel_values': tensor([[[-1.9809, -1.9809, -1.9809,  ..., -1.9809, -1.9809, -1.9809],
          [-1.9809, -1.9809, -1.9809,  ..., -1.9809, -1.9809, -1.9809],
          [-1.9809, -1.9809, -1.9809,  ..., -1.9809, -1.9809, -1.9809],
          ...,
          [ 0.7419,  0.6734,  0.6221,  ..., -0.1143, -0.1314, -0.1314],
          [ 0.8276,  0.6906,  0.7933,  ..., -0.2171, -0.1657, -0.2513],
          [ 0.9474,  0.7762,  0.8618,  ..., -0.3541, -0.1486, -0.2513]],
 
         [[-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          ...,
          [ 1.1856,  1.1331,  1.0630,  ...,  0.36

In [9]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["name"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [10]:
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred["predictions"], axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred["label_ids"])

In [19]:
batch_size = 256
lr_rate = 0.0008
num_epoch = 20
weight_decay = 0.05
label_smoothing_factor = 0.1

In [20]:
train_data_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn)
test_data_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, collate_fn=collate_fn)
val_data_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, collate_fn=collate_fn)

In [22]:
import torch
import wandb

model = load_model('./configs/models/resnext101_32x8d_ver1.0.0.toml')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

wandb.init(
    project='mineral-net',
    config={
        "learning_rate": lr_rate,
        "num_epoch": num_epoch,
        "batch_size": batch_size,
        "weight_decay": weight_decay,
    }
)

artifact = wandb.Artifact("config", type="config")
artifact.add_file("./configs/models/resnext101_32x8d_ver1.0.0.toml")
wandb.log_artifact(artifact)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr_rate, weight_decay=weight_decay)
model.to(device)
step = 0
for epoch in range(num_epoch):
    model.train()
    for batch in train_data_loader:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(pixel_values)
        loss = torch.nn.functional.cross_entropy(outputs, labels, label_smoothing=label_smoothing_factor)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()   
        wandb.log({"train_loss": loss.item()}, step=step)
        step += 1

    with torch.no_grad():
        model.eval()
        tot_loss = 0
        for batch in val_data_loader:
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(pixel_values)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            tot_loss += loss.item()
            
        wandb.log({"val_loss": tot_loss/len(val_data_loader)}, step=step)

    wandb.log({"epoch": epoch})

# Final testing phase with metrics
with torch.no_grad():
    model.eval()
    tot_loss = 0
    all_preds = []
    all_labels = []
    for batch in test_data_loader:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(pixel_values)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        tot_loss += loss.item()
        all_preds.append(outputs.cpu())
        all_labels.append(labels.cpu())
        
    wandb.log({"test_loss": tot_loss/len(test_data_loader)})
    wandb.log(compute_metrics({"predictions": all_preds, "label_ids": all_labels}))

wandb.finish()


Using cache found in C:\Users\Nech/.cache\torch\hub\pytorch_vision_main


train_loss,▁
train_loss,4.59437


wandb: WARNING Tried to log to step 51 that is less than the current step 52. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 102 that is less than the current step 103. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 153 that is less than the current step 154. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 204 that is less than the current step 205. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 255 that is less than the current step 256. Steps must be monotonically increasing, so this data will be ignored. See http

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11,) + inhomogeneous part.

In [53]:
wandb.log(compute_metrics({"predictions": outputs.cpu(), "label_ids": labels.cpu()}))

In [ ]:
torch.save(model.state_dict(), '101_2.pth')

In [21]:
wandb.finish()

train_loss,▁
train_loss,91.79338
